In [1]:
import sys
sys.path.append(str('/home/jupyter/SearchForecasting/'))
import pandas as pd
from configparser import ConfigParser
from repo.MyConfigParser import MyConfigParser
from repo.LoadData import DataLoader
from repo.BuildModel import GenerateModelFile, CreateModel
from repo.Accuracy import AccuracyCalculator, ProphetParameterSearch, PlotAccuracyCharts, MakeFuture
from repo.FileGenerator import *
from repo.Optimization import SavantPSO

In [2]:
updated_config = ConfigParser()
config_in_path = "/home/jupyter/SearchForecasting/Spain/June 2024 Update/config ups spain.ini"
updated_config.read(config_in_path)

getdict = MyConfigParser()
config_dict=getdict.GetDict(updated_config)
region = config_dict['CLIENTINFO']['country']
cutoff_month = config_dict['PROPHETSETTING']['cut_off_point']
base_start = config_dict['PROPHETSETTING']['reference_start_date1']
base_end = config_dict['PROPHETSETTING']['reference_end_date1']
periods = config_dict['PROPHETSETTING']['periods']
base_periods  = config_dict['PROPHETSETTING']['base_periods']

output_bucket = config_dict["FILEPATH"]["output_bucket"]
folder_name = config_dict["FILEPATH"]["folder_name"]

base_path = config_dict["FILEPATH"]["base_path"]

boundary_df_in_path = base_path + config_dict["FILEPATH"]["boundary_df_in_path"]
daily_df_out_path = base_path + config_dict["FILEPATH"]["daily_df_out_path"]
analytical_file_path = base_path + config_dict["FILEPATH"]["analytical_file_path"]
cuts_df_in_path = base_path + config_dict["FILEPATH"]["cuts_df_in_path"]
plot_name = base_path + config_dict["FILEPATH"]["plot_name"]
constraint_file_name = base_path + config_dict["FILEPATH"]["constraint_file_name"]
breakdown_ls = [config_dict['CLIENTINFO'][i] for i in [f'breakdown{i}' for i in range(1,5)] if config_dict['CLIENTINFO'][i] != None]

small_market_str = ''
pre_forecast_period_budget = float(config_dict['CLIENTINFO']['pre_forecast_period_budget'])
client_budget = float(config_dict['CLIENTINFO']['client_budget'])

save_date = config_dict['CLIENTINFO']['save_date']

In [3]:
# Get forecasters
bdDf = pd.read_csv(cuts_df_in_path)#.drop('Unnamed: 0', axis = 1)
cuts_df = bdDf.reset_index(drop = True)
cuts_df['forecaster_name_list'] =  cuts_df['market'] + ' ' + cuts_df['account_type'] + ' ' + cuts_df['funnel'] + '.pkl' #' 05.19.2021 Final.pkl'
cuts_df['periods'] = periods
cuts_df.drop(columns='Unnamed: 0', inplace=True)
forecasters_array = cuts_df.forecaster_name_list

# Get analytical file
analytical_file = pd.read_csv(analytical_file_path).drop('Unnamed: 0', axis = 1)
analytical_file['ds'] = pd.to_datetime(analytical_file['ds'])
analytical_file_check_version = analytical_file.copy()

# Get constrain file
constraint_file = pd.read_csv(constraint_file_name)

In [4]:
#constraint_file.loc[(constraint_file['Month'] == 'Nov') & (constraint_file['MinWeight'] == 0.8) & (constraint_file['MaxWeight'] == 1.2), ['MinWeight', 'MaxWeight']] = [1.1, 1.3]

In [5]:
constraint_file

,market,account_type,funnel,Month,Year,variable_splits,Constrain_Variable_Name,periods_for_acc_cal,changepoint_prior_scale,seasonality_prior_scale,...,competitor_cost_split,new_confirmed,power,growth,MAPE,period_AE,periods,cutoff_month,MinWeight,MaxWeight
0,Spain,Desktop,-,Apr,2024,Spain_Desktop_-_Apr_2024,cost,183,0.05,1.0,...,na,na,na,linear,22.22,8.62,251.0,2024-04-25,0.8,1.2
1,Spain,Desktop,-,May,2024,Spain_Desktop_-_May_2024,cost,183,0.05,1.0,...,na,na,na,linear,22.22,8.62,251.0,2024-04-25,0.8,1.2
2,Spain,Desktop,-,Jun,2024,Spain_Desktop_-_Jun_2024,cost,183,0.05,1.0,...,na,na,na,linear,22.22,8.62,251.0,2024-04-25,0.8,1.2
3,Spain,Desktop,-,Jul,2024,Spain_Desktop_-_Jul_2024,cost,183,0.05,1.0,...,na,na,na,linear,22.22,8.62,251.0,2024-04-25,0.8,1.2
4,Spain,Desktop,-,Aug,2024,Spain_Desktop_-_Aug_2024,cost,183,0.05,1.0,...,na,na,na,linear,22.22,8.62,251.0,2024-04-25,0.8,1.2
5,Spain,Desktop,-,Sep,2024,Spain_Desktop_-_Sep_2024,cost,183,0.05,1.0,...,na,na,na,linear,22.22,8.62,251.0,2024-04-25,0.8,1.2
6,Spain,Desktop,-,Oct,2024,Spain_Desktop_-_Oct_2024,cost,183,0.05,1.0,...,na,na,na,linear,22.22,8.62,251.0,2024-04-25,0.8,1.2
7,Spain,Desktop,-,Nov,2024,Spain_Desktop_-_Nov_2024,cost,183,0.05,1.0,...,na,na,na,linear,22.22,8.62,251.0,2024-04-25,0.8,0.9
8,Spain,Desktop,-,Dec,2024,Spain_Desktop_-_Dec_2024,cost,183,0.05,1.0,...,na,na,na,linear,22.22,8.62,251.0,2024-04-25,0.8,1.2
9,Spain,Mobile & Tablet,-,Apr,2024,Spain_Mobile_&_Tablet_-_Apr_2024,cost,183,0.10,10.0,...,na,na,na,linear,14.61,11.93,251.0,2024-04-25,0.8,1.2


In [6]:
today = '06.06.2024' #datetime.date.today().strftime("%m.%d.%Y")# 
fileName = base_path + f'Pos Result {today}.csv' #today
pos_df = pd.read_csv(fileName).drop('Unnamed: 0', axis = 1)
# pos_df = pd.DataFrame({'variable_splits':analytical_file.columns[19:37],'pos': 1})
# pos_df.pos = 1
pos_df

,variable_splits,pos
0,Spain_Desktop_-_Apr_2024,1.142977
1,Spain_Desktop_-_May_2024,1.199898
2,Spain_Desktop_-_Jun_2024,0.881294
3,Spain_Desktop_-_Jul_2024,1.039992
4,Spain_Desktop_-_Aug_2024,0.816963
5,Spain_Desktop_-_Sep_2024,1.132500
6,Spain_Desktop_-_Oct_2024,1.156240
7,Spain_Desktop_-_Nov_2024,0.818900
8,Spain_Desktop_-_Dec_2024,1.016773
9,Spain_Mobile_&_Tablet_-_Apr_2024,0.876780


In [7]:
resultPSO = SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights = pos_df.pos.to_numpy()

daily_simulated_forecast = resultPSO.ResultGenerator(inputWeights)

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  774048.0 difference between planned and forecast X:  0.0
yhat: 74233.0
analytical_spend_latest_period: 163077.561231284
client_budget - analytical_spend_latest_period: 0


In [8]:
resultPSO = SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights_base = np.ones(constraint_file.shape[0])

daily_simulated_forecast_base = resultPSO.ResultGenerator(inputWeights_base)

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  774048.0 difference between planned and forecast X:  0.0
yhat: 73299.0
analytical_spend_latest_period: 163077.56123128402
client_budget - analytical_spend_latest_period: 0


In [9]:
forecasted_daily_df = daily_simulated_forecast[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
forecasted_daily_df['month'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
forecasted_daily_df['year'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%Y').astype(str)

forecasted_daily_df = forecasted_daily_df.rename(columns = { 'yhat': 'OptimalConversionQuantity', 'X': 'OptimalSpendValue'})
forecasted_daily_df['OptimalSpendValue'] = forecasted_daily_df['OptimalSpendValue'].round(2)

forecasted_daily_df['variable_splits'] = forecasted_daily_df['market'] + '_' + forecasted_daily_df['account_type']+'_'+forecasted_daily_df['funnel']+'_'+forecasted_daily_df['month']+'_'+forecasted_daily_df['year']
print(forecasted_daily_df.OptimalSpendValue.sum(), forecasted_daily_df.OptimalConversionQuantity.sum())
forecasted_daily_df

774047.55 74233.0


,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits
0,Spain,Desktop,-,2024-04-25,2046.0,29900.13,Apr,2024,Spain_Desktop_-_Apr_2024
1,Spain,Desktop,-,2024-04-26,2763.0,41669.74,Apr,2024,Spain_Desktop_-_Apr_2024
2,Spain,Desktop,-,2024-04-27,307.0,5080.35,Apr,2024,Spain_Desktop_-_Apr_2024
3,Spain,Desktop,-,2024-04-28,220.0,3519.80,Apr,2024,Spain_Desktop_-_Apr_2024
4,Spain,Desktop,-,2024-04-29,797.0,9595.54,Apr,2024,Spain_Desktop_-_Apr_2024
...,...,...,...,...,...,...,...,...,...
497,Spain,Mobile & Tablet,-,2024-12-27,11.0,113.54,Dec,2024,Spain_Mobile & Tablet_-_Dec_2024
498,Spain,Mobile & Tablet,-,2024-12-28,6.0,113.54,Dec,2024,Spain_Mobile & Tablet_-_Dec_2024
499,Spain,Mobile & Tablet,-,2024-12-29,9.0,113.54,Dec,2024,Spain_Mobile & Tablet_-_Dec_2024
500,Spain,Mobile & Tablet,-,2024-12-30,17.0,52.92,Dec,2024,Spain_Mobile & Tablet_-_Dec_2024


In [10]:
base_daily_df = daily_simulated_forecast_base[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
base_daily_df['month'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
base_daily_df['year'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%Y').astype(str)
base_daily_df['X'] = base_daily_df['X'].round(2)
base_daily_df = base_daily_df.rename(columns = { 'X': 'BaseSpend', 'yhat': 'BaseConversionQuantity'})

print(base_daily_df.BaseSpend.sum(), base_daily_df.BaseConversionQuantity.sum())
base_daily_df

774047.49 73299.0


,market,account_type,funnel,ds,BaseConversionQuantity,BaseSpend,month,year
0,Spain,Desktop,-,2024-04-25,2034.0,29717.97,Apr,2024
1,Spain,Desktop,-,2024-04-26,2747.0,41415.87,Apr,2024
2,Spain,Desktop,-,2024-04-27,305.0,5049.40,Apr,2024
3,Spain,Desktop,-,2024-04-28,219.0,3498.35,Apr,2024
4,Spain,Desktop,-,2024-04-29,793.0,9537.08,Apr,2024
...,...,...,...,...,...,...,...,...
497,Spain,Mobile & Tablet,-,2024-12-27,11.0,102.00,Dec,2024
498,Spain,Mobile & Tablet,-,2024-12-28,6.0,102.00,Dec,2024
499,Spain,Mobile & Tablet,-,2024-12-29,9.0,102.00,Dec,2024
500,Spain,Mobile & Tablet,-,2024-12-30,17.0,47.54,Dec,2024


In [11]:
forecast_n_base_df = pd.merge(forecasted_daily_df, base_daily_df, how = 'left', on = ['market', 'account_type', 'funnel', 'ds', 'month', 'year'])
forecast_n_base_df['SavedDate'] = datetime.date.today().strftime("%Y-%m-%d")
forecast_n_base_df = forecast_n_base_df.rename(columns=lambda x: x[0:1].upper() + x[1:])
forecast_n_base_df = forecast_n_base_df.drop(['Variable_splits'], axis = 1)
# forecast_n_base_df.insert(3, 'WeekStart', [ i for i in forecast_n_base_df.Ds.apply(lambda x: x - pd.offsets.Week(weekday=4) )])
forecast_n_base_df.insert(0, 'Region', region)

forecast_n_base_df[forecast_n_base_df.Ds>='2024-06-01'].sum(numeric_only=1)

OptimalConversionQuantity     32602.00
OptimalSpendValue            163077.47
BaseConversionQuantity        32624.00
BaseSpend                    163077.50
dtype: float64

In [12]:
daily_simulated_forecast[((daily_simulated_forecast['account_type'] == 'Desktop')
                   | (daily_simulated_forecast['account_type'] == 'Mobile & Tablet'))]

,region_x,market,funnel,account_type,ds,X,Month,Year,Spain_Desktop_-_Apr_2024,Spain_Desktop_-_Aug_2024,...,Spain_Mobile_&_Tablet_-_Oct_2024,Spain_Mobile_&_Tablet_-_Sep_2024,Weekend,region_y,revenue,cost,conversion,AOV,ROAS,yhat
0,EMEA,Spain,-,Desktop,2024-04-25,29900.129417,Apr,2024,29900.129417,0.0,...,0.0,0.0,1,EMEA,2.266402e+06,8355484.22,71899,31.522022,0.271247,2046.0
1,EMEA,Spain,-,Desktop,2024-04-26,41669.742048,Apr,2024,41669.742048,0.0,...,0.0,0.0,1,EMEA,2.266402e+06,8355484.22,71899,31.522022,0.271247,2763.0
2,EMEA,Spain,-,Desktop,2024-04-27,5080.351954,Apr,2024,5080.351954,0.0,...,0.0,0.0,1,EMEA,2.266402e+06,8355484.22,71899,31.522022,0.271247,307.0
3,EMEA,Spain,-,Desktop,2024-04-28,3519.798412,Apr,2024,3519.798412,0.0,...,0.0,0.0,1,EMEA,2.266402e+06,8355484.22,71899,31.522022,0.271247,220.0
4,EMEA,Spain,-,Desktop,2024-04-29,9595.540794,Apr,2024,9595.540794,0.0,...,0.0,0.0,0,EMEA,2.266402e+06,8355484.22,71899,31.522022,0.271247,797.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,EMEA,Spain,-,Mobile & Tablet,2024-12-27,113.536465,Dec,2024,0.000000,0.0,...,0.0,0.0,1,EMEA,6.444920e+04,536891.88,2800,23.017573,0.120041,11.0
498,EMEA,Spain,-,Mobile & Tablet,2024-12-28,113.536465,Dec,2024,0.000000,0.0,...,0.0,0.0,1,EMEA,6.444920e+04,536891.88,2800,23.017573,0.120041,6.0
499,EMEA,Spain,-,Mobile & Tablet,2024-12-29,113.536465,Dec,2024,0.000000,0.0,...,0.0,0.0,1,EMEA,6.444920e+04,536891.88,2800,23.017573,0.120041,9.0
500,EMEA,Spain,-,Mobile & Tablet,2024-12-30,52.917332,Dec,2024,0.000000,0.0,...,0.0,0.0,0,EMEA,6.444920e+04,536891.88,2800,23.017573,0.120041,17.0


In [13]:
forecast_n_base_df.loc[((daily_simulated_forecast['account_type'] == 'Desktop')
                   | (daily_simulated_forecast['account_type'] == 'Mobile & Tablet'))]

,Region,Market,Account_type,Funnel,Ds,OptimalConversionQuantity,OptimalSpendValue,Month,Year,BaseConversionQuantity,BaseSpend,SavedDate
0,Spain,Spain,Desktop,-,2024-04-25,2046.0,29900.13,Apr,2024,2034.0,29717.97,2024-06-06
1,Spain,Spain,Desktop,-,2024-04-26,2763.0,41669.74,Apr,2024,2747.0,41415.87,2024-06-06
2,Spain,Spain,Desktop,-,2024-04-27,307.0,5080.35,Apr,2024,305.0,5049.40,2024-06-06
3,Spain,Spain,Desktop,-,2024-04-28,220.0,3519.80,Apr,2024,219.0,3498.35,2024-06-06
4,Spain,Spain,Desktop,-,2024-04-29,797.0,9595.54,Apr,2024,793.0,9537.08,2024-06-06
...,...,...,...,...,...,...,...,...,...,...,...,...
497,Spain,Spain,Mobile & Tablet,-,2024-12-27,11.0,113.54,Dec,2024,11.0,102.00,2024-06-06
498,Spain,Spain,Mobile & Tablet,-,2024-12-28,6.0,113.54,Dec,2024,6.0,102.00,2024-06-06
499,Spain,Spain,Mobile & Tablet,-,2024-12-29,9.0,113.54,Dec,2024,9.0,102.00,2024-06-06
500,Spain,Spain,Mobile & Tablet,-,2024-12-30,17.0,52.92,Dec,2024,17.0,47.54,2024-06-06


In [14]:
forecast_n_base_df = forecast_n_base_df.rename(columns={'Account_type': 'device_segment'})
forecast_n_base_df = forecast_n_base_df.drop('Funnel', axis=1)

In [15]:
forecast_n_base_df[['Year']] = forecast_n_base_df[['Year']].astype(int)
forecast_n_base_df["SavedDate"] = pd.to_datetime(forecast_n_base_df["SavedDate"])
forecast_n_base_df['SavedDate'] = forecast_n_base_df['SavedDate'].dt.date


In [16]:
forecast_n_base_df[forecast_n_base_df.Ds>='2024-06-01'].to_csv(base_path + 'Spain Output.csv')

In [17]:
forecast_n_base_df.query("Ds >= '2024-06-01'", inplace=True)

# Construct a BigQuery client object.
bqclient = bigquery.Client()

target_table_name = 'cost_by_device_' + region + save_date
table_id = "ups-analytics.forecast_optimization." + target_table_name

# job_config.
job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")

load_job = bqclient.load_table_from_dataframe(
    forecast_n_base_df, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = bqclient.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 428 rows.


table = bqclient.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)